In [ ]:
## Subscribes entities from frost then publishes them to Kafka

In [1]:
from __future__ import print_function
import sqlite3
import paho.mqtt.client as mqtt
import csv
import datetime
import os
import json
import requests
import sys
from kafka import KafkaProducer
from kafka import SimpleProducer, KafkaClient
#basepath = '/root/sftp/volumes/saqn/
import Helper

In [3]:
kafkaHome = Helper.Helper.Kafkas["dev01"]

frostHome = Helper.Helper.Frosts["dev01"]

def on_connect(client, userdata, flags, rc):
     print('Starting to log data.')
     client.subscribe("v1.0/Things")
     client.subscribe("v1.0/Datastreams")
     client.subscribe("v1.0/Locations")
     client.subscribe("v1.0/HistoricalLocations")
     client.subscribe("v1.0/Sensors")
     client.subscribe("v1.0/ObservedProperties")
     client.subscribe("v1.0/FeaturesOfInterest")
     client.subscribe("v1.0/Observations")

     if rc==0:
        print('connection established')

     else:
        print('Bad connection (return code = %d).' % rc)

def on_message(client, userdata, msg):
    payload = msg.payload
    topic = msg.topic.split('/')[-1]

    print("Receiving new message with topic: " + topic)
    #print(topic)
    #print(payload)

    data = json.loads(payload.decode("utf-8"))
    if(topic == "Observations"):
        iot_id = data['Datastream']['@iot.id']
    else:
        iot_id = data.get('@iot.id')
    #print(iot_id)
    producer.send(topic, key= str(iot_id), value=data)#msg.payload)
    producer.flush()


In [4]:
kafka = KafkaClient( kafkaHome +':9092')

#producer = SimpleProducer(kafka, async=True)
#producer.send_messages(b'my-topic', b'async message')
producer = KafkaProducer(bootstrap_servers=kafkaHome + ':9092',value_serializer=lambda v: json.dumps(v).encode('utf-8'), compression_type='gzip')

#producer.send('fizzbuzz', key= 'uid', value={'foo': 'bar'})

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(frostHome, 1883, 60)
client.loop_forever()

Starting to log data.
connection established


KeyboardInterrupt: 